# Pandas - "Excel" for Python

In [3]:
import pandas as pd

In [6]:
df = pd.read_csv("plz_verzeichnis_v2.csv", delimiter=';')

In [11]:
df.head()

,REC_ART,ONRP,BFSNR,PLZ_TYP,POSTLEITZAHL,PLZ_ZZ,GPLZ,ORTBEZ18,ORTBEZ27,KANTON,SPRACHCODE,SPRACHCODE_ ABW,BRIEFZ_DURCH,GILT_AB_DAT,PLZ_BRIEFZUST,PLZ_COFF,Geo Shape,Geokoordinaten
0,1,111,5586,80,1000,7,1000,Lausanne St-Paul,Lausanne St-Paul,VD,2,NaN,130,1993-09-28,100060,NaN,NaN,NaN
1,1,118,5586,80,1000,17,1000,Lausanne 17,Lausanne 17,VD,2,NaN,130,1986-05-21,100060,NaN,NaN,NaN
2,1,119,5586,80,1000,19,1000,Lausanne 19,Lausanne 19,VD,2,NaN,130,1993-09-28,100060,NaN,NaN,NaN
3,1,120,5586,80,1000,20,1000,Lausanne Sévelin,Lausanne Sévelin,VD,2,NaN,130,1993-09-06,100060,NaN,NaN,NaN
4,1,126,5586,10,1000,26,1000,Lausanne 26,Lausanne 26,VD,2,NaN,130,1986-05-21,100060,J,"{""type"": ""MultiPolygon"", ""coordinates"": [[[[6....","46.5556661916,6.69622535374"


In [7]:
df.columns

Index(['REC_ART', 'ONRP', 'BFSNR', 'PLZ_TYP', 'POSTLEITZAHL', 'PLZ_ZZ', 'GPLZ',
       'ORTBEZ18', 'ORTBEZ27', 'KANTON', 'SPRACHCODE', 'SPRACHCODE_ ABW',
       'BRIEFZ_DURCH', 'GILT_AB_DAT', 'PLZ_BRIEFZUST', 'PLZ_COFF', 'Geo Shape',
       'Geokoordinaten'],
      dtype='object')

In [23]:
# Which cantons exist in the DataFrame?
cantons = df['KANTON'].unique()
cantons

array(['VD', 'FR', 'GE', 'BE', 'VS', 'NE', 'SO', 'JU', 'BL', 'BS', 'AG',
       'LU', 'OW', 'ZG', 'NW', 'UR', 'SZ', 'TI', 'GR', 'SG', 'ZH', 'SH',
       'TG', 'GL', 'AR', 'AI', 'FL', 'IT', 'DE'], dtype=object)

**Note:** an "array" is returned, this is a *numpy* array, not a Python list!

In [30]:
cantons = list(cantons)
cantons.sort()

print(", ".join(cantons))  # Hint: String join

AG, AI, AR, BE, BL, BS, DE, FL, FR, GE, GL, GR, IT, JU, LU, NE, NW, OW, SG, SH, SO, SZ, TG, TI, UR, VD, VS, ZG, ZH


## DataFrame Analysis

In [36]:
df.describe()

,REC_ART,ONRP,BFSNR,PLZ_TYP,POSTLEITZAHL,PLZ_ZZ,GPLZ,SPRACHCODE,SPRACHCODE_ ABW,BRIEFZ_DURCH,PLZ_BRIEFZUST
count,5249.0,5249.000000,5249.000000,5249.000000,5249.000000,5249.00000,5249.000000,5249.000000,38.000000,5249.000000,5249.000000
mean,1.0,4061.358544,3328.607925,27.485235,5001.626215,6.62069,5000.500857,1.444085,1.657895,5273.463136,500438.230711
std,0.0,2586.192474,2160.856367,23.822119,2703.935789,18.07825,2703.473621,0.639741,0.534047,2765.492308,269718.811091
min,1.0,104.000000,1.000000,10.000000,1000.000000,0.00000,1000.000000,1.000000,1.000000,114.000000,100010.000000
25%,1.0,2009.000000,1065.000000,10.000000,2500.000000,0.00000,2500.000000,1.000000,1.000000,2730.000000,250060.000000
50%,1.0,3746.000000,3407.000000,20.000000,4715.000000,0.00000,4715.000000,1.000000,2.000000,6237.000000,480060.000000
75%,1.0,5633.000000,5323.000000,20.000000,7310.000000,2.00000,7310.000000,2.000000,2.000000,7666.000000,743060.000000
max,1.0,10743.000000,7301.000000,80.000000,9658.000000,93.00000,9658.000000,3.000000,3.000000,10711.000000,965700.000000


In [38]:
df['POSTLEITZAHL'].describe()

count    5249.000000
mean     5001.626215
std      2703.935789
min      1000.000000
25%      2500.000000
50%      4715.000000
75%      7310.000000
max      9658.000000
Name: POSTLEITZAHL, dtype: float64

# Classifier: Postal code to canton

Can we learn which postal codes belong to which canton?

* Input X = postal code
* Output Y = canton

In [81]:
learn_cols = ['POSTLEITZAHL']
predict_col = 'KANTON'

In [86]:
df_plz = df[learn_cols + [predict_col]].copy()

In [87]:
df_plz[predict_col] = df_plz[predict_col].astype('category')

In [88]:
df_plz.sample(n=5)

,POSTLEITZAHL,KANTON
4374,6839,TI
1265,6086,BE
4337,6622,TI
1177,5036,AG
3278,7741,GR


## Train/test data

We must convert string values into something a machine can learn - labels.

Here we must "One hot" encoding. 0 or 1 for each possible value.

In [104]:
# Encode cantons (strings) as unique labels
canton_labels = pd.get_dummies(df_plz[predict_col])

In [195]:
df_plz_labelled = pd.concat([df_plz[learn_cols], canton_labels], axis=1)

In [196]:
df_plz_labelled.sample(n=5)

,POSTLEITZAHL,AG,AI,AR,BE,BL,BS,DE,FL,FR,...,SH,SO,SZ,TG,TI,UR,VD,VS,ZG,ZH
3603,1063,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
842,2350,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3550,9534,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
488,3654,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1174,5024,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Separate into training and test data

* We learn on one part of the data
* We test on a different part of the data

Why? Prevent learning all the examples by heart!

In [113]:
# scikit learn has machine learning utilities and examples for learning
from sklearn.model_selection import train_test_split

In [174]:
X, y = df_plz_labelled.iloc[:,:1], df_plz_labelled.iloc[:,1:]

In [201]:
X.columns

Index(['POSTLEITZAHL'], dtype='object')

In [232]:
label_names = y.columns.values
label_names

array(['AG', 'AI', 'AR', 'BE', 'BL', 'BS', 'DE', 'FL', 'FR', 'GE', 'GL',
       'GR', 'IT', 'JU', 'LU', 'NE', 'NW', 'OW', 'SG', 'SH', 'SO', 'SZ',
       'TG', 'TI', 'UR', 'VD', 'VS', 'ZG', 'ZH'], dtype=object)

In [203]:
# Separate data into train and test parts
# Note: Fixed random seed for reproducibility

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [204]:
print("Training")
print("X:", x_train.shape)
print("Y:", y_train.shape)
print("Test")
print("X:", x_test.shape)
print("Y:", y_test.shape)

Training
X: (4199, 1)
Y: (4199, 29)
Test
X: (1050, 1)
Y: (1050, 29)


In [205]:
x_train.head()

,POSTLEITZAHL
3538,9493
4274,6330
3837,9545
1127,4563
2544,2024


In [206]:
y_train.head()

,AG,AI,AR,BE,BL,BS,DE,FL,FR,GE,...,SH,SO,SZ,TG,TI,UR,VD,VS,ZG,ZH
3538,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4274,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3837,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1127,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2544,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Train our classifier

Let's teach our "black box" which postal codes go with which canton. Later we will ask different examples!

In [212]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier()

In [208]:
classifier.fit(x_train, y_train)

DecisionTreeClassifier()

## Test

What has the classifier learned?

In [209]:
predictions = classifier.predict(x_test)

In [214]:
predictions.shape == y_test.shape

True

In [217]:
pd.DataFrame(predictions)

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1046,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1047,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1048,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [221]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [220]:
accuracy_score(y_test, predictions)

0.9695238095238096

In [227]:
y_labels = y_test.values.argmax(axis=1)
pred_labels = predictions.argmax(axis=1)

In [233]:
def labelToCanton(label_names, label_id):
    return label_names[label_id]

In [235]:
labelToCanton(label_names, 0)

'AG'

In [268]:
result = x_test.copy()
result['true_label'] = y_labels
result['true_canton'] = result['true_label'].apply(lambda x: labelToCanton(label_names, x))
result['output_label'] = pred_labels
result['output_canton'] = result['output_label'].apply(lambda x: labelToCanton(label_names, x))
result['correct'] = result.apply(lambda row: row['true_label'] == row['output_label'], axis=1)

result.drop(columns=['true_label', 'output_label'], inplace=True)

result.sample(n=10)

,POSTLEITZAHL,true_canton,output_canton,correct
2791,3944,VS,VS,True
3334,8305,ZH,ZH,True
3456,8881,SG,SG,True
4357,6717,TI,TI,True
2259,3360,BE,BE,True
129,1423,VD,VD,True
2005,6900,TI,TI,True
288,3238,BE,BE,True
591,1200,GE,GE,True
5085,3041,BE,BE,True


In [269]:
result[result['correct'] == False]

,POSTLEITZAHL,true_canton,output_canton,correct
4759,9532,TG,SG,False
1798,9403,SG,AR,False
2415,1563,FR,VD,False
5233,8920,ZH,AG,False
3663,8905,AG,ZH,False
2837,4143,SO,BL,False
1825,9536,SG,TG,False
1066,4117,BL,SO,False
152,1543,VD,FR,False
3461,8905,AG,ZH,False
